In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import datetime

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Froms
from src.gcpUtils.auth import getCredentials
from src.gcpUtils.bigQuery import pandasToBq, tableToPandas
from src.gcpUtils.google_storage_manager import *


cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")

# Detalhamento do Processamento (Análise de Ressuprimento por Filial)

# O que é computado por Filial
- Detecção de Dias sem Ressuprimento:
    - Converte a coluna de data para datetime.
    - Considera que "tem ressuprimento" se a coluna RESSUPRIR for diferente de 0. Caso contrário (0 ou nulo), é considerado "sem ressuprimento".

- Identificação de Sequências (Gaps):
    - Agrupa os dados por FILIAL e ordena por data.
    - Identifica o início e fim de sequências consecutivas onde não houve necessidade de ressuprimento.
    - Calcula a duração em dias (dias_sem_ressuprimento) para cada sequência identificada.

- Classificação e Status:
    - Status do Período: "EM ANDAMENTO" (se a sequência vai até a última data disponível) ou "FINALIZADO".
    - Classificação do Período: "SUPERIOR A 30 DIAS" ou "ATE 30 DIAS".
    - Status da Filial: Se a filial tiver pelo menos um período classificado como "> 30 dias", ela recebe a tag "FILIAL COM PERÍODO > 30 DIAS". Caso contrário, "FILIAL SEM PERÍODO > 30 DIAS".

- Lista filtrada final: O script lê o arquivo mestre de filiais e exporta um novo arquivo contendo apenas as filiais que não tiveram períodos sem ressuprimento superiores a 30 dias.

- Critério de Exclusão: O JSON final exclui qualquer filial que tenha ficado mais de 30 dias consecutivos sem ressuprimento ("SUPERIOR A 30 DIAS").

In [13]:
query = """
SELECT
    FILIAL,
    DATA,
    RESSUPRIR
FROM `planejamento-animale-292719.checklists_rollout.ANIMALE_checklist`
WHERE DATE(DATA) > DATE('2025-01-15')
  AND DATE(DATA) < DATE('2025-02-06')

"""

df_raw = tableToPandas(query, 'planejamento-animale-292719', cred)
df_raw.columns = df_raw.columns.str.upper()



In [14]:
def detectar_periodos_sem_ressuprimento(
    df,
    date_col='DATA',
    filial_col='FILIAL',
    resupply_col='RESSUPRIR'
):
    """
    Recebe um DataFrame com (pelo menos) colunas: FILIAL, DATA, RESSUPRIR.
    Retorna um DataFrame com a mesma estrutura lógica da query SQL:
      - por FILIAL: ID_PERIODO, inicio_sem_ressuprimento, fim_sem_ressuprimento,
        dias_sem_ressuprimento, CLASSIFICACAO, STATUS
      - e um STATUS_FILIAL indicando se teve período > 30 dias (FILIAL COM PERÍODO > 30 DIAS / FILIAL SEM PERÍODO > 30 DIAS)
    Observações:
      - date_col pode ser string; a função converte para datetime.
      - resupply_col é a coluna usada para determinar "ressuprimento"; a condição é (resupply_col != 0).
    """

    # --- 1) Validações e conversões ---
    if filial_col not in df.columns:
        raise ValueError(f"Coluna {filial_col} não encontrada no DataFrame.")
    if date_col not in df.columns:
        raise ValueError(f"Coluna {date_col} não encontrada no DataFrame.")
    if resupply_col not in df.columns:
        raise ValueError(f"Coluna {resupply_col} não encontrada no DataFrame.")

    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])

    df['__tem_ressuprimento__'] = df[resupply_col].fillna(0).ne(0).astype(int)

    base = (
        df
        .groupby([filial_col, date_col], as_index=False)
        .agg(produtos_com_ressuprimento = ('__tem_ressuprimento__', 'sum'))
    )

    # --- 3) status_filial: sem_ressuprimento = 1 quando produtos_com_ressuprimento == 0 ---
    base['sem_ressuprimento'] = (base['produtos_com_ressuprimento'] == 0).astype(int)

    # --- 4) marcacao_inicio: detecta início de uma sequência de sem_ressuprimento por filial ---
    base = base.sort_values([filial_col, date_col])
    base['sem_ressuprimento_lag'] = base.groupby(filial_col)['sem_ressuprimento'].shift(1).fillna(0).astype(int)

    base['inicio_periodo'] = (
        ((base['sem_ressuprimento'] == 1) & (base['sem_ressuprimento_lag'] == 0))
        .astype(int)
    )

    # --- 5) grupos: cria periodo_id somando cumulativamente inicio_periodo por filial ---
    base['periodo_id_cum'] = base.groupby(filial_col)['inicio_periodo'].cumsum()
    grupos = base[ base['sem_ressuprimento'] == 1 ].copy()

    # --- 6) resumo: agrupa por FILIAL + periodo_id_cum para obter inicio/fim/dias ---
    if not grupos.empty:
        resumo = (
            grupos
            .groupby([filial_col, 'periodo_id_cum'], as_index=False)
            .agg(
                inicio_sem_ressuprimento = (date_col, 'min'),
                fim_sem_ressuprimento = (date_col, 'max'),
            )
        )
        resumo['dias_sem_ressuprimento'] = (
            (resumo['fim_sem_ressuprimento'] - resumo['inicio_sem_ressuprimento']).dt.days + 1
        ).astype(int)

        resumo = resumo.rename(columns={'periodo_id_cum': 'ID_PERIODO'})

        # --- 7) classificado: STATUS (EM ANDAMENTO / FINALIZADO) e CLASSIFICACAO (>30 / <=30) ---
        ultima_data_por_filial = (
            base.groupby(filial_col)[date_col].max().rename('ultima_data').reset_index()
        )
        resumo = resumo.merge(ultima_data_por_filial, on=filial_col, how='left')

        resumo['STATUS'] = np.where(
            resumo['fim_sem_ressuprimento'].dt.floor('D') == resumo['ultima_data'].dt.floor('D'),
            'EM ANDAMENTO',
            'FINALIZADO'
        )

        resumo['CLASSIFICACAO'] = np.where(
            resumo['dias_sem_ressuprimento'] > 30,
            'SUPERIOR A 30 DIAS',
            'ATE 30 DIAS'
        )

        resumo = resumo.drop(columns=['ultima_data'])
    else:
        resumo = pd.DataFrame(
            columns=[filial_col, 'ID_PERIODO', 'inicio_sem_ressuprimento', 'fim_sem_ressuprimento',
                     'dias_sem_ressuprimento', 'STATUS', 'CLASSIFICACAO']
        )

    # --- 8) agrupado: por filial, se teve algum periodo > 30 dias ---
    if not resumo.empty:
        agrupado = (
            resumo
            .assign(teve_periodo_maior_30 = (resumo['CLASSIFICACAO'] == 'SUPERIOR A 30 DIAS').astype(int))
            .groupby(filial_col, as_index=False)
            .agg(teve_periodo_maior_30 = ('teve_periodo_maior_30', 'max'))
        )
    else:
        filiais_presentes = base[filial_col].drop_duplicates().reset_index(drop=True)
        agrupado = pd.DataFrame({filial_col: filiais_presentes, 'teve_periodo_maior_30': 0})

    resultado = agrupado.merge(resumo, on=filial_col, how='left')

    resultado['STATUS_FILIAL'] = np.where(
        resultado['teve_periodo_maior_30'] == 1,
        'FILIAL COM PERÍODO > 30 DIAS',
        'FILIAL SEM PERÍODO > 30 DIAS'
    )

    cols_final = [
        filial_col, 'STATUS_FILIAL', 'ID_PERIODO',
        'inicio_sem_ressuprimento', 'fim_sem_ressuprimento',
        'dias_sem_ressuprimento', 'CLASSIFICACAO', 'STATUS'
    ]
    for c in cols_final:
        if c not in resultado.columns:
            resultado[c] = pd.NA

    resultado = resultado[cols_final].sort_values([filial_col, 'ID_PERIODO']).reset_index(drop=True)

    for dcol in ['inicio_sem_ressuprimento', 'fim_sem_ressuprimento']:
        if dcol in resultado.columns:
            resultado[dcol] = pd.to_datetime(resultado[dcol]).dt.date

    return resultado


df_prod_desc = detectar_periodos_sem_ressuprimento(
    df_raw,
    date_col='DATA',
    filial_col='FILIAL',
    resupply_col='RESSUPRIR'
)

In [ ]:
local_path_in = os.path.join(os.getcwd(), '../data')
os.makedirs(local_path_in, exist_ok=True)

file_path_in = os.path.join(local_path_in, 'ressuprimento_filiais_python.xlsx')
df_prod_desc.to_excel(file_path_in, index=False)

print(f"Arquivo salvo em {file_path_in}")

Arquivo salvo em c:\Users\Leonardo Verissimo\repositorio_final\Project_AZZAS2154\notebooks\../data\ressuprimento_filiais.xlsx


In [16]:
# Filiais que possuem pelo menos um período > 30 dias
filiais_superior_30 = (
    df_prod_desc.loc[df_prod_desc["CLASSIFICACAO"] == "SUPERIOR A 30 DIAS", "FILIAL"]
    .drop_duplicates()
    .tolist()
)

# Todas as filiais presentes no dataframe
todas_filiais = df_prod_desc["FILIAL"].unique().tolist()
filiais_nao_superior_30 = [f for f in todas_filiais if f not in filiais_superior_30]

dict_filiais_superior_30 = {"SUPERIOR_30": filiais_superior_30}
dict_filiais_nao_superior_30 = {"NAO_SUPERIOR_30": filiais_nao_superior_30}

qtd_filiais_superior_30 = len(filiais_superior_30)
qtd_filiais_nao_superior_30 = len(filiais_nao_superior_30)
qtd_total_filiais = len(todas_filiais)

print("Qtd filiais superior 30 dias:", qtd_filiais_superior_30)
print("Qtd filiais <= 30 dias:", qtd_filiais_nao_superior_30)


Qtd filiais superior 30 dias: 0
Qtd filiais <= 30 dias: 60


In [18]:
import json
# pasta atual
print(os.getcwd())

json_path = '../dados/todas_filiais.json'
output_path = '../dados/filiais_inferior_30.json'

# Escolha qual dicionário usar:
dict_filiais = dict_filiais_nao_superior_30
# dict_filiais = dict_filiais_superior_30

def filtrar_filiais_por_dicionario(json_path, dict_filiais, output_path):
    # Extrai todas as filiais do dicionário
    nomes_filiais = []
    for valor in dict_filiais.values():
        if isinstance(valor, list):
            nomes_filiais.extend(valor)
        else:
            nomes_filiais.append(valor)

    nomes_filiais = [nome.upper().strip() for nome in nomes_filiais]
    with open(json_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)

    # Filtra apenas filiais desejadas
    filtradas = [
        filial for filial in dados
        if isinstance(filial, dict)
        and str(filial.get("FILIAL", "")).strip().upper() in nomes_filiais
    ]

    # Salva JSON filtrado
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(filtradas, f, indent=4, ensure_ascii=False)
    print(f"Arquivo salvo em: {os.path.abspath(output_path)}")
    return filtradas

filiais_filtradas = filtrar_filiais_por_dicionario(
    json_path=json_path,
    dict_filiais=dict_filiais,
    output_path=output_path
)

print(f"Total de filiais exportadas: {len(filiais_filtradas)}")


c:\Users\Leonardo Verissimo\repositorio_final\Project_AZZAS2154\notebooks
Arquivo salvo em: c:\Users\Leonardo Verissimo\repositorio_final\Project_AZZAS2154\dados\filiais_inferior_30.json
Total de filiais exportadas: 60
